In [12]:
#import libraries
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import KFold, train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

import numpy as np

In [10]:
from sklearn.model_selection import train_test_split

# dataset class
class NutritionData:
    
    def __init__(self, data_path=None):
        self.GOOD_OUTPUT =['Animal Products', 'Animal fats', 'Meat', 'Cereals - Excluding Beer','Vegetables','Vegetal Products','Vegetable Oils', 'Oilcrops' ]
        self.NOT_IPMORTANT_COLUMNS  = ["Country", 'Unit (all except Population)', 'Population', 
                                      'Obesity', 'Undernourished', 'Confirmed', 'Deaths','Recovered', 'Active',
                                      'Alcoholic Beverages',
           'Aquatic Products, Other', 'Eggs',
           'Fish, Seafood', 'Fruits - Excluding Wine',
           'Milk - Excluding Butter', 'Offals','Pulses', 'Spices',
           'Starchy Roots', 'Stimulants', 'Sugar Crops', 'Sugar & Sweeteners',
           'Treenuts',
           'Miscellaneous']
        
        if data_path != None:
            self.data_path = data_path
            dataframe = pd.read_csv(self.data_path)
            dataframe = self.__replaceNaN(dataframe, replaceby="mostfrq")

            ##Standard Scaler 
            #NOT_IPMORTANT_COLUMNS = ["Country", 'Unit (all except Population)', 'Population', 
            #                       'Obesity', 'Undernourished', 'Confirmed', 'Deaths','Recovered', 'Active']

            

            clm = dataframe.columns.drop(self.NOT_IPMORTANT_COLUMNS)

            dataframe = self.__replace_char(dataframe, "Undernourished", char=r'^<')
            scaled_df, self.scaler   = self.__scale(dataframe, clm, scaler="mm")
            scaled_df = scaled_df.sample(frac=1,random_state=1).reset_index(drop=True)

            self.dataset = scaled_df
        
    
        # Importan Methodes
    # Replace the nan vale with mean, median, or most frequent value
    def __replaceNaN(self, dataframe, replaceby = "mean"):
        if (replaceby == "mean"):
            return dataframe.fillna(dataframe.mean())

        elif (replaceby == "median"):
            return dataframe.fillna(dataframe.median())

        elif (replaceby == "mostfrq"):
            return dataframe.fillna(dataframe.mode().iloc[0])

        else:return dataframe.dropna()
        
        

    # Replce any nonnumaric char from a column.    
    def __replace_char(self, dataframe, collumn, char=r'^<'):
        
        dataframe[collumn] =  dataframe[collumn].replace(regex=char, value='')
        dataframe[collumn] = pd.to_numeric(dataframe[collumn],errors='coerce')
        return dataframe



    # Scale the data using standard scaler or max min scaler. Menstion the names to scale. 
    def __scale(self, dataframe, coulumns, scaler="ss"):
        
        if (scaler == "mm"):
            from sklearn.preprocessing import MinMaxScaler
            scaler = MinMaxScaler()
            scaler.fit(dataframe[coulumns])
            dataframe[coulumns] = scaler.fit_transform(dataframe[coulumns])
            return dataframe, scaler
        elif (scaler == "ss"):
            from sklearn.preprocessing import StandardScaler
            scaler = StandardScaler()
            scaler.fit(dataframe[coulumns])
            datqframe[coulumns] = scaler.fit_transform(dataframe[coulumns])
            return dataframe, scaler

        else:return dataframe, scaler





# Applying PCA to find out maximum variance 

# Protein Data

In [13]:
dataobj = NutritionData(data_path="../dash_app/data/Protein_Supply_Quantity_Data.csv")
# X_ori = Protein_Supply_DF.iloc[:, 1:25].values
# y_ori = Protein_Supply_DF.iloc[:, 24].values

column_names=dataobj.dataset.columns[1:25]

X=dataobj.dataset.iloc[:, 1:25].values
y=dataobj.dataset.iloc[:, 24].values

X = StandardScaler().fit_transform(X)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

X_train_ori = X_train
X_test_ori = X_test

# Applying PCA
from sklearn.decomposition import PCA
for i in range (8,0,-1):
    pca = PCA(n_components = i)
    X_train = pca.fit_transform(X_train_ori)
    X_test = pca.transform(X_test_ori)
    explained_variance = pca.explained_variance_ratio_
    print(explained_variance)
    n_pcs= pca.components_.shape[0]
    most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
    most_important_names = [column_names[most_important[i]] for i in range(n_pcs)]
    #print(most_important)
    print(most_important_names)
    print("------")

[0.29823065 0.08947424 0.0677232  0.06341445 0.05755083 0.05566106
 0.05075314 0.0424012 ]
['Vegetal Products', 'Fish, Seafood', 'Alcoholic Beverages', 'Sugar & Sweeteners', 'Fruits - Excluding Wine', 'Spices', 'Stimulants', 'Treenuts']
------
[0.29823065 0.08947424 0.0677232  0.06341445 0.05755083 0.05566106
 0.05075314]
['Vegetal Products', 'Fish, Seafood', 'Alcoholic Beverages', 'Sugar & Sweeteners', 'Fruits - Excluding Wine', 'Spices', 'Stimulants']
------
[0.29823065 0.08947424 0.0677232  0.06341445 0.05755083 0.05566106]
['Vegetal Products', 'Fish, Seafood', 'Alcoholic Beverages', 'Sugar & Sweeteners', 'Fruits - Excluding Wine', 'Spices']
------
[0.29823065 0.08947424 0.0677232  0.06341445 0.05755083]
['Vegetal Products', 'Fish, Seafood', 'Alcoholic Beverages', 'Sugar & Sweeteners', 'Fruits - Excluding Wine']
------
[0.29823065 0.08947424 0.0677232  0.06341445]
['Vegetal Products', 'Fish, Seafood', 'Alcoholic Beverages', 'Sugar & Sweeteners']
------
[0.29823065 0.08947424 0.06772

# Fat Consumption Data

In [14]:
dataobj = NutritionData(data_path="../dash_app/data/Fat_Supply_Quantity_Data.csv")
# X_ori = Protein_Supply_DF.iloc[:, 1:25].values
# y_ori = Protein_Supply_DF.iloc[:, 24].values

column_names=dataobj.dataset.columns[1:25]

X=dataobj.dataset.iloc[:, 1:25].values
y=dataobj.dataset.iloc[:, 24].values

X = StandardScaler().fit_transform(X)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

X_train_ori = X_train
X_test_ori = X_test

# Applying PCA
from sklearn.decomposition import PCA
for i in range (8,0,-1):
    pca = PCA(n_components = i)
    X_train = pca.fit_transform(X_train_ori)
    X_test = pca.transform(X_test_ori)
    explained_variance = pca.explained_variance_ratio_
    print(explained_variance)
    n_pcs= pca.components_.shape[0]
    most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
    most_important_names = [column_names[most_important[i]] for i in range(n_pcs)]
    #print(most_important)
    print(most_important_names)
    print("------")

[0.2697507  0.10698497 0.08134613 0.07966437 0.06265716 0.05657478
 0.04968203 0.04745227]
['Animal Products', 'Vegetable Oils', 'Vegetables', 'Fruits - Excluding Wine', 'Miscellaneous', 'Spices', 'Animal fats', 'Fruits - Excluding Wine']
------
[0.2697507  0.10698497 0.08134613 0.07966437 0.06265716 0.05657478
 0.04968203]
['Animal Products', 'Vegetable Oils', 'Vegetables', 'Fruits - Excluding Wine', 'Miscellaneous', 'Spices', 'Animal fats']
------
[0.2697507  0.10698497 0.08134613 0.07966437 0.06265716 0.05657478]
['Animal Products', 'Vegetable Oils', 'Vegetables', 'Fruits - Excluding Wine', 'Miscellaneous', 'Spices']
------
[0.2697507  0.10698497 0.08134613 0.07966437 0.06265716]
['Animal Products', 'Vegetable Oils', 'Vegetables', 'Fruits - Excluding Wine', 'Miscellaneous']
------
[0.2697507  0.10698497 0.08134613 0.07966437]
['Animal Products', 'Vegetable Oils', 'Vegetables', 'Fruits - Excluding Wine']
------
[0.2697507  0.10698497 0.08134613]
['Animal Products', 'Vegetable Oils', 

# Energy Data

In [ ]:
dataobj = NutritionData(data_path="../dash_app/data/Fat_Supply_Quantity_Data.csv")
# X_ori = Protein_Supply_DF.iloc[:, 1:25].values
# y_ori = Protein_Supply_DF.iloc[:, 24].values

column_names=dataobj.dataset.columns[1:25]

X=dataobj.dataset.iloc[:, 1:25].values
y=dataobj.dataset.iloc[:, 24].values

X = StandardScaler().fit_transform(X)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

X_train_ori = X_train
X_test_ori = X_test

# Applying PCA
from sklearn.decomposition import PCA
for i in range (8,0,-1):
    pca = PCA(n_components = i)
    X_train = pca.fit_transform(X_train_ori)
    X_test = pca.transform(X_test_ori)
    explained_variance = pca.explained_variance_ratio_
    print(explained_variance)
    n_pcs= pca.components_.shape[0]
    most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
    most_important_names = [column_names[most_important[i]] for i in range(n_pcs)]
    #print(most_important)
    print(most_important_names)
    print("------")